In [1]:
import sys

In [2]:
sys.path.insert(1, '../')

In [3]:
from utils.dataset import EHRDataset
from utils.dataset import EHRDatasetPredictionTask
from model.tokenizer import EHRTokenizer
import pytorch_pretrained_bert as Bert
from torch.utils.data import DataLoader
from model.model import *
from utils.config import BertConfig
from model.trainer import PatientTrajectoryPredictor
import pytorch_lightning as pl

In [4]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [5]:
path = '../processing/next_visit'

In [6]:
data = pd.read_parquet(path)

In [7]:
data.head()

,subject_id,icd_code,age,alcohol_abuse,tobacco_abuse,ndc,hadm_id,medication_label,gender,icd_labels
0,10236661,"[[34690, 42731, 42781, 4019, 311, V5861, V4501...",[80.0],[0],[0],"[[245004101, 51079082120, 56017275, 93435693, ...",[21942970],"[406055262, 63481068706, 51079007320, 51079069...",F,[4280]
1,10334189,"[[5770, 0380, 99591, 5761, 99741, 42490, E8786...",[91.0],[0],[0],"[[338011704, 245004101, 409672924, 206886202, ...",[23784349],"[51079025520, 62584071301, 245004101, 61314003...",F,[4280]
2,10387770,"[[7218, 5845, 2762, 27900, 2760, 3572, 5859, 4...","[82.0, 83.0]","[0, 0]","[0, 0]","[[904777261, 456066270, 338004903, 55390000401...","[25294592, 25124191]","[50268085415, 338004931, 54327099, 409915801, ...",M,"[42832, 4280]"
3,10627407,"[[43820, 43889, 72989, 43819, 78830, 78720, 56...","[82.0, 82.0, 83.0, 84.0, 85.0, 85.0]","[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]","[[63323026201, 904516561, 74434113, 6473900, 9...","[25789023, 21419302, 21187686, 23697251, 26313...","[409189001, 904404073, 60505011300, 71041724, ...",F,"[42822, 4280]"
4,10661086,"[[27739, 42833, 4257, 20300, 5849, 4160, 4280,...",[60.0],[0],[0],"[[256015201, 51079080120, 63020004901, 5107900...",[29124943],"[517571025, 338070341, 409125830, 338601048, 6...",M,"[42840, 4280]"


In [8]:
#c=['4280', '4281', '42820', '42821', '42822', '42823', '42830', '42831' ,'42832', '42833', '42840', '42841' ,'42842', '42843', 
#'4289']

In [9]:
def filter_codes(codes):
    newcodes=[]
    for code in codes:
        if str(code) in c:
            newcodes.append(code)
            
    return newcodes

In [10]:
#data['icd_labels'] = data['icd_label'].apply(lambda x: filter_codes(x))

In [11]:
#data

In [12]:
#data.drop(columns='icd_label', inplace=True)

In [13]:
#data.to_parquet('../processing/next_visit')

In [14]:
#label_voc = {}

#labels = data['icd_labels'].tolist()

In [15]:
#alllabels = []

In [16]:
#alllabels = []
#for labellist in labels:
#    #print(labellist)
 #   for label in labellist:
 #       if label not in label_voc:
 #           label_voc[label] = len(label_voc)
  #  alllabels.extend(labellist)            

In [17]:
#np.save('../processing/label_voc.npy', np.array(alllabels))

In [18]:
#len(label_voc)

In [19]:
data.rename(columns={'icd_labels':'label'}, inplace=True)

In [52]:
global_params = {
    'max_seq_len': 32,
    'max_age': 110,
    'month': 1,
    'age_symbol': None,
    'min_visit': 5,
    'gradient_accumulation_steps': 1
}

optim_param = {
    'lr': 3e-5,
    'warmup_proportion': 0.1,
    'weight_decay': 0.01
}

train_params = {
    'batch_size': 32,
    'use_cuda': True,
    'max_len_seq': global_params['max_seq_len'],
    'device': 'cuda' #change this to run on cuda #'cuda:0'
}

In [53]:
tokenizer = EHRTokenizer()

In [54]:
Dset = EHRDatasetPredictionTask(data, max_len=train_params['max_len_seq'], tokenizer=tokenizer, prediction_task='visit')
trainload = DataLoader(dataset=Dset, batch_size=train_params['batch_size'], shuffle=True)

Epoch 4:  53%|█████████████████████████                      | 1357/2543 [03:09<02:45,  7.15it/s, loss=0.174, v_num=126]


In [55]:
model_config = {
    'vocab_size': len(tokenizer.getVoc('code').keys()), # number of disease + symbols for word embedding
    'hidden_size': 200, # word embedding and seg embedding hidden size
    'seg_vocab_size': 2, # number of vocab for seg embedding
    'age_vocab_size': len(tokenizer.getVoc('age').keys()), # number of vocab for age embedding,
    'gender_vocab_size': 3,
    'max_position_embeddings': train_params['max_len_seq'], # maximum number of tokens
    'hidden_dropout_prob': 0.1, # dropout rate
    'num_hidden_layers': 2, # number of multi-head attention layers required
    'num_attention_heads': 4, # number of attention heads
    'attention_probs_dropout_prob': 0.1, # multi-head attention dropout rate
    'intermediate_size': 300, # the size of the "intermediate" layer in the transformer encoder
    'hidden_act': 'gelu', # The non-linear activation function in the encoder and the pooler "gelu", 'relu', 'swish' are supported
    'initializer_range': 0.02, # parameter weight initializer range
}

In [56]:
conf = BertConfig(**model_config)
model = BertMultiLabelPrediction(conf, num_labels=len(tokenizer.getVoc('label').keys()))

In [57]:
def load_model(path, model):
    # load pretrained model and update weights
    pretrained_dict = torch.load(path)
    model_dict = model.state_dict()
    # 1. filter out unnecessary keys
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    # 2. overwrite entries in the existing state dict
    model_dict.update(pretrained_dict)
    # 3. load the new state dict
    model.load_state_dict(model_dict)
    return model

PATH = "checkpoint2"
mode = load_model(PATH, model)

In [66]:
optim_config = {
    'lr': 3e-5,
    'warmup_proportion': 0.1,
    'weight_decay': 0.01
}

In [67]:
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
params = list(model.named_parameters())
optimizer_grouped_parameters = [
    {'params': [p for n, p in params if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in params if any(nd in n for nd in no_decay)], 'weight_decay': 0.01}
]

optim = Bert.optimization.BertAdam(optimizer_grouped_parameters,
                                   lr=optim_config['lr'],
                                   warmup=optim_config['warmup_proportion'])

t_total value of -1 results in schedule not being applied


In [68]:
voc = tokenizer.getVoc('label').values()

In [69]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=list(voc))
mlb.fit([[each] for each in list(voc)])

MultiLabelBinarizer(classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])

In [70]:
patienttrajectory = PatientTrajectoryPredictor(model, optim, optim_param, train_objective='visit', metrics=True, binarizer=mlb)

In [71]:
tensorboarddir = '../logs/'

In [72]:
trainer = pl.Trainer(
            max_epochs=20, 
            gpus=1,
            logger=pl.loggers.TensorBoardLogger(save_dir=tensorboarddir),
            callbacks=[pl.callbacks.progress.TQDMProgressBar()], 
            progress_bar_refresh_rate=1,
            weights_summary=None, # Can be None, top or full
            num_sanity_val_steps=10,
        )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [73]:
trainer.fit(
    patienttrajectory, 
    train_dataloader=trainload,
);

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 6:  20%|██████████▏                                       | 65/318 [00:20<01:20,  3.14it/s, loss=0.183, v_num=130]